<img src="http://datapark.io/img/logo.png" alt="datpark" width="35%" align="right" border="0"><br>

# blaze &ndash; Data Blending and Analysis

``blaze`` allows Python users a familiar interface to **query data living in diverse data storage systems**.
Cf. <a href="http://blaze.pydata.org/" target="_blank">http://blaze.pydata.org/</a>.

In [1]:
import blaze as bz

## Simple Example 

The first example constructs a ``blaze.Data`` object from native Python objects.

In [2]:
t = bz.Data([('Henry', 'boy', 8),
              ('Lilli', 'girl', 14)],
            fields=['name', 'gender', 'age'])

In [3]:
t

,name,gender,age
0,Henry,boy,8
1,Lilli,girl,14


In [4]:
t[t.age > 10]

,name,gender,age
0,Lilli,girl,14


## Data from NumPy Array

Let us read data from an **in-memory NumPy ``ndarray`` object**.

In [5]:
import numpy as np

In [6]:
a = np.random.standard_normal((1000000, 5))
  # 1mn data rows, 5 columns

In [7]:
df = bz.DataFrame(a, columns=['f0', 'f1', 'f2', 'f3', 'f4'])
  # blaze DataFrame constructor

A look at the **data structure**.

In [8]:
df.head()

,f0,f1,f2,f3,f4
0,2.795037,-0.195276,-1.198247,-0.166165,-1.346425
1,0.659529,-1.684173,-0.447169,0.489202,1.271425
2,1.625823,-0.776000,-0.206676,1.525741,-2.359000
3,0.682128,0.167127,1.114394,-0.141836,1.104020
4,0.718311,-1.422499,-1.121962,-0.407740,0.028113


Data itself is stored as **NumPy ``ndarray`` object**.

In [9]:
df.values

array([[ 2.79503673, -0.19527603, -1.19824678, -0.16616534, -1.34642456],
       [ 0.65952903, -1.68417253, -0.44716858,  0.48920235,  1.27142457],
       [ 1.62582317, -0.77599993, -0.20667587,  1.52574141, -2.35900047],
       ..., 
       [ 0.47648798, -0.01352559, -0.04537713, -0.98707368, -0.10778052],
       [ 0.53932827,  0.88623521,  1.78712157, -0.95080939,  0.5013271 ],
       [ 1.23982146,  0.06834312, -1.19967816,  0.71556697, -0.2691043 ]])

## Data from CSV File

We generate first a **CSV file** using the random data from before.

In [10]:
path = 'data/'

In [11]:
%time df.to_csv(path + 'data.csv', index=False)

CPU times: user 3.3 s, sys: 96.3 ms, total: 3.4 s
Wall time: 3.4 s


Let us **read the data** with ``blaze``. Actually, we **only generate a view**.

In [12]:
%time csv = bz.CSV(path + 'data.csv')

CPU times: user 318 µs, sys: 36 µs, total: 354 µs
Wall time: 237 µs


In [13]:
%time t1 = bz.Data(csv)

CPU times: user 5.35 ms, sys: 8.19 ms, total: 13.5 ms
Wall time: 11 ms


Now, we can **work** with the data. Note, however, that iterating, slicing, etc. are **not** (yet) implemented. 

In [14]:
%time t1.count()

CPU times: user 1.4 ms, sys: 157 µs, total: 1.55 ms
Wall time: 1.07 ms


1000000

The **backend** is a **CSV object**. And a look at the **first 10 rows**.

In [15]:
t1.data

In [16]:
t1

,f0,f1,f2,f3,f4
0,2.795037,-0.195276,-1.198247,-0.166165,-1.346425
1,0.659529,-1.684173,-0.447169,0.489202,1.271425
2,1.625823,-0.776000,-0.206676,1.525741,-2.359000
3,0.682128,0.167127,1.114394,-0.141836,1.104020
4,0.718311,-1.422499,-1.121962,-0.407740,0.028113
5,1.192323,0.194672,-0.953438,1.803671,2.152515
6,0.815519,0.628659,-1.049453,0.162612,1.836330
7,0.134279,0.519261,0.106039,0.676105,0.683848
8,0.454850,3.165847,1.024735,-0.029155,-0.545055
9,-0.620911,-1.330088,-0.228976,-1.410431,0.806394


## Data from SQL

We now generate a **SQLite3 table** with the dummy data from before.

In [17]:
import sqlite3 as sq3

In [18]:
con = sq3.connect(path + 'data.sql')
try:
    con.execute('DROP TABLE numbers')
    # delete in case it exists
except:
    pass

We **write the data** into an appropriate table.

In [19]:
con.execute(
    'CREATE TABLE numbers (f0 real, f1 real, f2 real, f3 real, f4 real)'
    )

In [20]:
%time con.executemany('INSERT INTO numbers VALUES (?, ?, ?, ?, ?)', a)

CPU times: user 10 s, sys: 288 ms, total: 10.3 s
Wall time: 10.3 s


In [21]:
con.commit()

In [22]:
con.close()

Now **reading the data** with ``blaze`` (i.e. just generating a view).

In [23]:
%time t2 = bz.Data('sqlite:///%sdata.sql::numbers' % path)

CPU times: user 73.1 ms, sys: 32.4 ms, total: 105 ms
Wall time: 103 ms


The **schma** and **first 10 rows**.

In [24]:
t2.schema

dshape("{f0: ?float64, f1: ?float64, f2: ?float64, f3: ?float64, f4: ?float64}")

In [25]:
t2

,f0,f1,f2,f3,f4
0,2.795037,-0.195276,-1.198247,-0.166165,-1.346425
1,0.659529,-1.684173,-0.447169,0.489202,1.271425
2,1.625823,-0.776000,-0.206676,1.525741,-2.359000
3,0.682128,0.167127,1.114394,-0.141836,1.104020
4,0.718311,-1.422499,-1.121962,-0.407740,0.028113
5,1.192323,0.194672,-0.953438,1.803671,2.152515
6,0.815519,0.628659,-1.049453,0.162612,1.836330
7,0.134279,0.519261,0.106039,0.676105,0.683848
8,0.454850,3.165847,1.024735,-0.029155,-0.545055
9,-0.620911,-1.330088,-0.228976,-1.410431,0.806394


## Working with the blaze Objects

``blaze`` provides an **abstraction logic** for computations/queries.

In [26]:
ts = bz.TableSymbol('ts',
        '{f0: float64, f1: float64, f2: float64, f3: float64, f4: float64}')
  # generic table description -- independent of the target data structure
expr = ts[ts['f0'] + ts['f3'] > 2.5]['f1']
  # generic expression -- independent of the target data structure

The ``blaze`` compiler **specializes the generic objects** to different data structures.

In [27]:
%time np.array(bz.compute(expr, a))  # NumPy ndarray object

CPU times: user 44.3 ms, sys: 4.02 ms, total: 48.3 ms
Wall time: 46.2 ms


array([-0.19527603, -0.77599993,  0.194672  , ..., -0.53242345,
       -0.18615608, -0.57159508])

In [28]:
%time np.array(bz.compute(expr, df))  # DataFrame object

CPU times: user 78.9 ms, sys: 7.44 ms, total: 86.4 ms
Wall time: 39.2 ms


array([-0.19527603, -0.77599993,  0.194672  , ..., -0.53242345,
       -0.18615608, -0.57159508])

In [29]:
%time np.array(bz.compute(expr, csv))  # CSV file representation

CPU times: user 731 ms, sys: 36.4 ms, total: 767 ms
Wall time: 760 ms


array([-0.19527603, -0.77599993,  0.194672  , ..., -0.53242345,
       -0.18615608, -0.57159508])

In similar fashion, ``blaze`` allows **unified expression evaluations** for different backends (I).

In [30]:
%time t1[t1['f0'] + t1['f3'] > 2.5]['f1'].head()
  # table representation 1
  # from CSV

CPU times: user 2.81 ms, sys: 4.07 ms, total: 6.88 ms
Wall time: 4.76 ms


,f1
0,-0.195276
2,-0.776000
5,0.194672
14,-0.404383
15,-0.015254
20,-0.812343
35,0.529581
43,0.064834
45,1.708357
66,0.249707


In similar fashion, ``blaze`` allows **unified expression evaluations** for different backends (II).

In [31]:
%time t2[t2['f0'] + t2['f3'] > 2.5]['f1'].head()
  # table representation 2
  # from SQL database

CPU times: user 7.5 ms, sys: 4.24 ms, total: 11.7 ms
Wall time: 7.98 ms


,f1
0,-0.195276
1,-0.776000
2,0.194672
3,-0.404383
4,-0.015254
5,-0.812343
6,0.529581
7,0.064834
8,1.708357
9,0.249707


Typical **aggregational operations** work as well.

In [32]:
%time t1.f0.sum()

CPU times: user 925 µs, sys: 0 ns, total: 925 µs
Wall time: 688 µs


-729.5905064757135

In [33]:
%time t2.f3.max()

CPU times: user 1.95 ms, sys: 128 µs, total: 2.08 ms
Wall time: 1.44 ms


4.727507078613924

## Transforming Data Formats

If you work intensively with data sets, it might be beneficial to transform them once into **highly performant binary data formats (eg ``bcolz, HDF5``)**.

### Using bcolz as Data Store

In [34]:
%time bz.into(path + 'data.bcolz', path + 'data.csv')
  # natively done by blaze
  # cparams=bcolz.cparams(9) could be added
  # no effect here due to random floats

CPU times: user 1.23 s, sys: 233 ms, total: 1.47 s
Wall time: 1.47 s


ctable((1000000,), [('f0', '<f8'), ('f1', '<f8'), ('f2', '<f8'), ('f3', '<f8'), ('f4', '<f8')])
  nbytes: 38.15 MB; cbytes: 38.52 MB; ratio: 0.99
  cparams := cparams(clevel=5, shuffle=True, cname='blosclz')
  rootdir := 'data/data.bcolz'
[ (2.7950367268400003, -0.195276031717, -1.19824677715, -0.166165337727, -1.34642456002)
 (0.659529030918, -1.6841725251, -0.447168579005, 0.48920235286799996, 1.27142456629)
 (1.62582317394, -0.775999929303, -0.206675874211, 1.52574140747, -2.35900046897)
 ...,
 (0.476487979456, -0.0135255931171, -0.045377126415599996, -0.9870736760240001, -0.10778051901900002)
 (0.5393282742000001, 0.88623521354, 1.7871215666999998, -0.9508093945399999, 0.501327101411)
 (1.23982146314, 0.0683431163467, -1.1996781637799998, 0.715566973299, -0.269104298199)]

We can now connect to the ``bcolz`` **disk-based ``ctable`` object**.

In [35]:
import bcolz as bc

In [36]:
b = bc.ctable(rootdir=path + 'data.bcolz') 

Now, the power of ``bcolz`` for **numerical computations** can be played out.

In [37]:
%time nex = b.eval('sqrt(abs(f0)) + log(abs(f1))')

CPU times: user 183 ms, sys: 67.3 ms, total: 251 ms
Wall time: 138 ms


In [38]:
nex

carray((1000000,), float64)
  nbytes: 7.63 MB; cbytes: 7.45 MB; ratio: 1.02
  cparams := cparams(clevel=5, shuffle=True, cname='blosclz')
[ 0.03849516  1.33338829  1.02147486 ..., -3.61289051  0.61361684
 -1.56974173]

### Using HDF5

Similarly, we can use **``PyTables`` and ``HDF5``** as an efficient binary store.

In [39]:
import pandas as pd

In [40]:
%%time
con = sq3.connect(path + 'data.sql')
pd.HDFStore(path + 'data.h5')['sql'] = pd.read_sql('SELECT * FROM numbers', con)
  # simultaneously reading whole SQL table and writing it to HDF5 store
con.close()

CPU times: user 1.47 s, sys: 257 ms, total: 1.73 s
Wall time: 1.73 s


Now, data can be **efficiently** retrieved.

In [41]:
%%time
import seaborn as sns; sns.set()
%matplotlib inline
pd.HDFStore(path + 'data.h5')['sql'][::1000].cumsum().plot(figsize=(10, 5))
  # simultaneously reading data from HDF5 store and plotting it

ImportError: libgthread-2.0.so.0: cannot open shared object file: No such file or directory

## Cleaning Up

In [ ]:
!du -h $path*

In [ ]:
!ls -n $path*

In [ ]:
# cleaning up
!rm -r $path*

## Conclusion

**High performance (hardware-bound) I/O operations and highly efficient data blending and analytics are among Python's key strengths.**

<img src="http://datapark.io/img/logo.png" alt="datapark" width="35%" align="right" border="0"><br>

<a href="http://datapark.io" target="_blank">datapark.io</a> | <a href="http://twitter.com/dataparkio" target="_blank">@dataparkio</a> | <a href="mailto:team@datapark.io">team@datapark.io</a>